# DA Throttling Analysis on Base

**Context**
- Investigate two windows for DA throttling: May 27, 4:14 pm → May 29, 3:29 pm CDT (highlight May 28 ~9:05–9:35 am CDT) and July 31, 5:30–6:30 pm PT.
- Sampling: 10% random, seeded for the long CDT window; full fetch for the 1-hour PT window.

**Analysis**
- Analyze DA throttling events using time-series visualization to understand:
  - When throttling starts/ends
  - How DA utilization patterns change during throttling
  - Base fee patterns


# Step 1: Imports and Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Core imports
import sys
from pathlib import Path
import polars as pl
from datetime import datetime, timedelta
from collections import Counter
import matplotlib.pyplot as plt
import time
import numpy as np

# Add parent directory to path
sys.path.append(str(Path.cwd().parent))

# Import Jovian modules
from jovian_src.chain_config import (
    get_chain_display_name
)
from jovian_src.constants import (
    DEFAULT_DA_FOOTPRINT_GAS_SCALARS
)
from jovian_src.clickhouse_fetcher import (
    load_gas_limits,
    fetch_top_percentile_blocks,
    fetch_random_sample_blocks,
    get_gas_limit_for_date
)
from jovian_src.analysis_functions import (
    perform_jovian_analysis,
    generate_jovian_recommendation
)
from jovian_src.visualization_jovian import (
    generate_all_visualizations,
    plot_block_metric_distribution
)

print("✅ Modules loaded successfully")
print(f"✅ Available scalars: {DEFAULT_DA_FOOTPRINT_GAS_SCALARS}")

✅ Modules loaded successfully
✅ Available scalars: [160, 400, 600, 800]


# Step 2: Configuration

In [ ]:
# ============================================================================
# ANALYSIS CONFIGURATION - EDIT THESE PARAMETERS
# ============================================================================

# Chain selection
CHAIN = "base"  # Options: base, op

# Sampling method
SAMPLING_METHOD = "random"  # "top_percentile" or "random"
RANDOM_SEED = 42     # For reproducible random sampling
SAMPLE_FRACTION = 0.01  # For random sampling (1% = 0.01)

# Date range
START_DATE = "2025-05-27"
END_DATE = "2025-05-29"  # Inclusive

# Analysis parameters
DA_FOOTPRINT_GAS_SCALARS = DEFAULT_DA_FOOTPRINT_GAS_SCALARS  # [160, 400, 600, 800]
BLOCK_LIMIT = None   # None = all blocks for selected method
FORCE_REFRESH = False  # Set True to ignore cache and re-download

# Output settings
SAVE_RESULTS = True
SHOW_PLOTS = True

print("✅ Configuration loaded:")
print(f"   Chain: {CHAIN}")
print(f"   Method: {SAMPLING_METHOD}")
if SAMPLING_METHOD == "top_percentile":
    print(f"   Percentile: Top {100-PERCENTILE}%")
else:
    print(f"   Sample: {int(SAMPLE_FRACTION*100)}% random")
print(f"   Date range: {START_DATE} to {END_DATE}")